In [ ]:
!pip install transformers seqeval shap scikit-learn

In [3]:
import pandas as pd
import numpy as np
import torch
import transformers
import sys
import os

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 加载模型

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model_name = 'bert-base-uncased' # 预训练模型名字
tokenizer = transformers.BertTokenizer.from_pretrained(model_name)
model = transformers.BertForSequenceClassification.from_pretrained(model_name, num_labels=2) # 需要预测的类别数为2
os.chdir('/content/drive/MyDrive/ModelDebug/classifier/baseline')
model_weights = torch.load('./pytorch_model.bin')
model.load_state_dict(model_weights)
model.eval()
model.to(device)

cuda


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [21]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

# 读取CSV文件
df = pd.read_csv('./expand.csv')

# 获取文本数据列和标签列
texts = df['RequirementText'].tolist()
labels = df['NFR'].tolist()

# 将标签转换为整数类型
labels = [int(label) for label in labels]

# 创建真实标签列表和预测标签列表
true_labels = labels
predicted_labels = []

# 遍历数据集进行预测
for text in texts:
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    inputs = inputs.to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)
    predicted_label = torch.argmax(probabilities, dim=1).item()
    predicted_labels.append(predicted_label)

# 将真实标签和预测标签移动到相同的设备上
true_labels = torch.tensor(true_labels).to(device)
predicted_labels = torch.tensor(predicted_labels).to(device)

# 计算指标
precision = precision_score(true_labels.cpu(), predicted_labels.cpu())
recall = recall_score(true_labels.cpu(), predicted_labels.cpu())
f1 = f1_score(true_labels.cpu(), predicted_labels.cpu())

# 输出指标结果
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Precision: 0.6894736842105263
Recall: 0.9097222222222222
F1 Score: 0.7844311377245509
